In [1]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
import numpy as np
import pandas as pd
from skimage.restoration import denoise_wavelet
from sklearn import model_selection
from nexcsi import decoder

# Initial data

In [2]:
device = "raspberrypi"
raw_data_path = "./Data8_Train/room/Red"
processed_path = "./train_data"
classes = ["empty", "walk", "jump", "wave"]
activity_code = {
    "EMPTY": "1",
    "WALK": "4",
    "STAND": "3",
    "SIT": "5",
}
activity_vector = {
    "empty": [0, 0, 0, 1],
    "walk": [0, 0, 1, 0],
    "jump": [0, 1, 0, 0],
    "wave": [1, 0, 0, 0],
}
records_num = 150
samples_num = records_num * len(classes)
bandwidth = 40
carriers_num = 108
timestamps_num = 150
x = []
y = []

# Signal Filters

In [3]:
def median_absolute_deviation(x):
    """
    Returns the median absolute deviation from the window's median
    :param x: Values in the window
    :return: MAD
    """
    return np.median(np.abs(x - np.median(x)))

def hampel_d(ts, window_size=5, n=3, imputation=False):

    """
    Median absolute deviation (MAD) outlier in Time Series
    :param ts: a pandas Series object representing the timeseries
    :param window_size: total window size will be computed as 2*window_size + 1
    :param n: threshold, default is 3 (Pearson's rule)
    :param imputation: If set to False, then the algorithm will be used for outlier detection.
        If set to True, then the algorithm will also imput the outliers with the rolling median.
    :return: Returns the outlier indices if imputation=False and the corrected timeseries if imputation=True
    """

    if type(ts) != pd.Series:
        raise ValueError("Timeserie object must be of tyme pandas.Series.")

    if type(window_size) != int:
        raise ValueError("Window size must be of type integer.")
    else:
        if window_size <= 0:
            raise ValueError("Window size must be more than 0.")

    if type(n) != int:
        raise ValueError("Window size must be of type integer.")
    else:
        if n < 0:
            raise ValueError("Window size must be equal or more than 0.")

    # Copy the Series object. This will be the cleaned timeserie
    ts_cleaned = ts.copy()

    # Constant scale factor, which depends on the distribution
    # In this case, we assume normal distribution
    k = 1.4826

    rolling_ts = ts_cleaned.rolling(window_size*2, center=True)
    rolling_median = rolling_ts.median().fillna(method='bfill').fillna(method='ffill')
    rolling_sigma = k*(rolling_ts.apply(median_absolute_deviation).fillna(method='bfill').fillna(method='ffill'))

    outlier_indices = list(
        np.array(np.where(np.abs(ts_cleaned - rolling_median) >= (n * rolling_sigma))).flatten())

    if imputation:
        ts_cleaned[outlier_indices] = rolling_median[outlier_indices]
        return ts_cleaned

    return outlier_indices



def hampel_filter_light(data, window_size=3, n_sigma=3):
    n = len(data)
    filtered = np.zeros(n)
    for i in range(n):
        lower = max(0, i - window_size)
        upper = min(n, i + window_size)
        x = data[lower:upper]
        median = np.median(x)
        deviation = np.abs(x - median)
        MAD = np.median(deviation)
        threshold = n_sigma * MAD
        if np.abs(data[i] - median) > threshold:
            filtered[i] = median
        else:
            filtered[i] = data[i]
    return filtered

# Readers and processors

In [4]:
def read_pcap(label, idx):
  samples_r = decoder(device).read_pcap(raw_data_path + '/' + label + '/R_' + label + '_00' + activity_code[label] + '_P150_' + str(idx).zfill(4) + '.pcap', bandwidth=bandwidth)

  return decoder(device).unpack(samples_r['csi'], zero_nulls=False)

def read_pcap_from(path):
    samples_r = decoder(device).read_pcap(path, bandwidth=bandwidth)

    return decoder(device).unpack(samples_r['csi'], zero_nulls=False)

def read_csv(label, idx):
  path = processed_path + '/' + label + '/' + label + '_' + str(idx + 1).zfill(4) + '.csv'
  return np.genfromtxt(path, delimiter=',')

def proccess_csi(csi):
  csi = np.delete(csi, csi.dtype.metadata['nulls'] + csi.dtype.metadata['pilots'], axis=1)
  csi = np.abs(csi)

  for i in range(len(csi)):
    csi[i] = list(hampel_d(pd.Series(csi[i]), 3, imputation=True))

  csi = denoise_wavelet(csi, wavelet='sym6', mode='soft', wavelet_levels=3, method='BayesShrink', rescale_sigma='True')

  return csi

def proccess_csi_light(csi):
  csi = np.delete(csi, csi.dtype.metadata['nulls'] + csi.dtype.metadata['pilots'], axis=1)
  csi = np.abs(csi)

  for i in range(len(csi)):
    csi[i] = hampel_filter_light(csi[i])

  csi = denoise_wavelet(csi, wavelet='sym6', mode='soft', wavelet_levels=3, method='BayesShrink', rescale_sigma='True')

  return csi

def read_and_process(path, separator = 9000):
    csi = read_pcap_from(path)
    csi = proccess_csi(csi)
    csi = csi[:separator]

    return np.array(np.split(csi.copy(), int(separator / timestamps_num)))

# Read dataset Data from CSV (preprocessed)

In [5]:
def read_csv(label, idx):
  path = processed_path + '/' + label + '/' + label + '_' + str(idx + 1).zfill(3) + '.csv'
  return np.genfromtxt(path, delimiter=',')

for label in classes:
  for i in range(0, records_num):
    y.append(label)
    x.append(read_csv(label, i))

for i in range(len(y)):
  y[i] = activity_vector[y[i]]

# Or Read dataset Data from raw .pcap (and preprocess it)

In [143]:
for label in classes:
  for i in range(1, records_num + 1):
    y.append(label)
    x.append(read_pcap(label, i))

for i in range(len(y)):
  y[i] = activity_vector[y[i]]

for i in range(len(x)):
  x[i] = proccess_csi(x[i])

for i in range(len(classes)):
  for ri in range(0, records_num):
    label = classes[i]
    idx = records_num*i + ri
    path = processed_path + '/' + label + '/' + label + '_' + str(ri + 1).zfill(4) + '.csv'
    np.savetxt(path, np.asarray(x[idx]), delimiter=",")

KeyError: 'empty'

# Generate train and test data

In [8]:
y1 = np.array(y.copy())
x1 = np.array(x.copy())

x_train, x_test, y_train, y_test = model_selection.train_test_split(x1, y1, test_size=0.2)

[[1 0 0 0]
 [0 1 0 0]
 [1 0 0 0]
 ...
 [0 0 1 0]
 [0 1 0 0]
 [0 0 0 1]]


In [35]:
opt = Adam(learning_rate=0.0001)

# Define the model architecture
model = Sequential()

# Add an input layer with the specified input shape
model.add(LSTM(units=125, input_shape=(150, 108), return_sequences=True))
model.add(Dropout(0.2))

# Add a second LSTM layer
model.add(LSTM(units=100))
model.add(Dropout(0.2))

# Add four fully connected network layers with relu activation
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=8, activation='relu'))

# Add the classification layer with softmax activation
model.add(Dense(units=4, activation='softmax'))

# Compile the model with specified loss function and optimizer
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


In [36]:
# Train the model with the specified number of epochs and batch size
model.fit(x_train, y_train, epochs=200, batch_size=32, validation_data=(x_test, y_test))

# Evaluate the model on the test set
score = model.evaluate(x_test, y_test, batch_size=32)

# Print the test accuracy
print("Test accuracy:", score[1])

Epoch 1/200
15/15 [==============================] - 4s 175ms/step - loss: 1.3822 - accuracy: 0.2771 - val_loss: 1.3973 - val_accuracy: 0.1917
Epoch 2/200
15/15 [==============================] - 2s 142ms/step - loss: 1.3819 - accuracy: 0.2708 - val_loss: 1.3953 - val_accuracy: 0.2167
Epoch 3/200
15/15 [==============================] - 2s 141ms/step - loss: 1.3771 - accuracy: 0.2729 - val_loss: 1.3913 - val_accuracy: 0.2000
Epoch 4/200
15/15 [==============================] - 2s 142ms/step - loss: 1.3740 - accuracy: 0.2562 - val_loss: 1.3862 - val_accuracy: 0.2583
Epoch 5/200
15/15 [==============================] - 2s 140ms/step - loss: 1.3658 - accuracy: 0.2896 - val_loss: 1.3715 - val_accuracy: 0.2833
Epoch 6/200
15/15 [==============================] - 2s 141ms/step - loss: 1.3655 - accuracy: 0.3063 - val_loss: 1.3866 - val_accuracy: 0.2583
Epoch 7/200
15/15 [==============================] - 2s 141ms/step - loss: 1.3540 - accuracy: 0.3333 - val_loss: 1.3789 - val_accuracy: 0.2250

In [29]:
model.save('./models/lstm_o.keras')

In [5]:
model = load_model('./models/lstm_o.keras')

# Processing and testing actual data from the room

In [14]:
packet_wave = read_and_process('./test/wave.pcap')
packet_walk = read_and_process('./test/walk.pcap')
packet_jump = read_and_process('./test/jump.pcap')
packet_empty = read_and_process('./test/empty.pcap')

In [15]:
model.predict(packet_empty)

2/2 [==============================] - 0s 33ms/step


array([[1.00963689e-06, 1.04924981e-02, 2.54350714e-03, 9.86962974e-01],
       [8.57307076e-01, 6.73609525e-02, 5.45369983e-02, 2.07949504e-02],
       [9.56937075e-01, 2.09024195e-02, 1.91920437e-02, 2.96850665e-03],
       [1.24893591e-01, 2.42506936e-01, 1.18422054e-01, 5.14177322e-01],
       [9.10891175e-01, 3.98599580e-02, 4.15918492e-02, 7.65708974e-03],
       [9.50963438e-01, 2.16226075e-02, 2.41292734e-02, 3.28475889e-03],
       [9.28931713e-01, 3.10628787e-02, 3.41153145e-02, 5.89018641e-03],
       [9.46153343e-01, 2.58114040e-02, 2.39649191e-02, 4.07030154e-03],
       [9.62516665e-01, 1.93745736e-02, 1.54430410e-02, 2.66570877e-03],
       [9.57832396e-01, 2.11029928e-02, 1.80965252e-02, 2.96799745e-03],
       [9.51555967e-01, 2.56933719e-02, 1.91188920e-02, 3.63173499e-03],
       [9.53241825e-01, 2.35375520e-02, 1.95219517e-02, 3.69869731e-03],
       [9.18734353e-03, 5.08141339e-01, 4.76641715e-01, 6.02959888e-03],
       [2.43016332e-01, 2.42836908e-01, 1.45997152e

In [11]:
model.predict(packet_wave)

2/2 [==============================] - 0s 37ms/step


array([[0.32370362, 0.31142524, 0.26523185, 0.09963924],
       [0.3175016 , 0.29446137, 0.25502643, 0.13301055],
       [0.2638508 , 0.24664845, 0.37068588, 0.11881492],
       [0.37742108, 0.1896247 , 0.30812183, 0.12483234],
       [0.30501863, 0.27849606, 0.29363185, 0.12285345],
       [0.82997394, 0.06373736, 0.07002258, 0.0362661 ],
       [0.63440335, 0.18647622, 0.10468692, 0.0744335 ],
       [0.33967894, 0.34293634, 0.12022132, 0.19716343],
       [0.2816546 , 0.34880725, 0.2045532 , 0.16498497],
       [0.27559614, 0.20833649, 0.4200614 , 0.09600585],
       [0.7292721 , 0.12233413, 0.08331327, 0.06508051],
       [0.2961744 , 0.18331821, 0.42727536, 0.09323207],
       [0.6909223 , 0.14478666, 0.09460926, 0.06968174],
       [0.5651567 , 0.22372092, 0.10842707, 0.10269531],
       [0.6624264 , 0.17692184, 0.08915633, 0.07149537],
       [0.49591926, 0.26753893, 0.1151256 , 0.12141616],
       [0.3494325 , 0.17875418, 0.38622722, 0.08558612],
       [0.7835809 , 0.08143532,

In [12]:
model.predict(packet_walk)

2/2 [==============================] - 0s 37ms/step


array([[0.44975525, 0.23948915, 0.19126551, 0.11949013],
       [0.42482537, 0.2725803 , 0.21300717, 0.08958714],
       [0.39971069, 0.25883192, 0.21305169, 0.1284057 ],
       [0.41902572, 0.2558328 , 0.19152409, 0.13361734],
       [0.40761298, 0.23814614, 0.22038525, 0.13385561],
       [0.25139794, 0.32300496, 0.27716377, 0.14843333],
       [0.5207796 , 0.20493653, 0.12892827, 0.14535561],
       [0.3992982 , 0.25248057, 0.19862808, 0.1495931 ],
       [0.37962127, 0.24019578, 0.2119013 , 0.1682816 ],
       [0.41778916, 0.24318433, 0.22275896, 0.11626759],
       [0.5458268 , 0.15807937, 0.23095626, 0.06513759],
       [0.37899223, 0.295453  , 0.20871347, 0.11684135],
       [0.23426935, 0.383652  , 0.22625546, 0.1558232 ],
       [0.5264997 , 0.24781048, 0.09868243, 0.12700741],
       [0.3242163 , 0.31006873, 0.18600644, 0.1797085 ],
       [0.3975006 , 0.24839059, 0.20744969, 0.14665908],
       [0.4125955 , 0.26413757, 0.21373567, 0.10953132],
       [0.7022562 , 0.09706607,

In [13]:
model.predict(packet_jump)

2/2 [==============================] - 0s 43ms/step


array([[0.8359353 , 0.04230146, 0.0842602 , 0.03750303],
       [0.8542182 , 0.03524317, 0.08044513, 0.03009342],
       [0.7963921 , 0.06007487, 0.08312363, 0.06040948],
       [0.86207634, 0.03054426, 0.07924587, 0.0281335 ],
       [0.84945863, 0.03585526, 0.07594498, 0.03874115],
       [0.85211   , 0.03242475, 0.07781545, 0.03764979],
       [0.81222045, 0.04438248, 0.09309618, 0.05030084],
       [0.89113915, 0.02418866, 0.06074667, 0.02392556],
       [0.8816172 , 0.03015577, 0.06682513, 0.0214019 ],
       [0.7723219 , 0.07149473, 0.08528293, 0.07090045],
       [0.812346  , 0.04784256, 0.08659967, 0.05321181],
       [0.7622188 , 0.0716893 , 0.10476804, 0.06132391],
       [0.85014105, 0.03965206, 0.07207339, 0.03813352],
       [0.8913319 , 0.02552658, 0.0609456 , 0.02219588],
       [0.7998556 , 0.06554238, 0.08569609, 0.04890587],
       [0.8204845 , 0.0516321 , 0.08507801, 0.04280543],
       [0.8011846 , 0.06156729, 0.08864123, 0.04860693],
       [0.8761184 , 0.02470079,

In [50]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy import signal

# Load the dataset
data = np.array(x.copy())
labels = np.array(y.copy())

# Preprocess the data
# Resample the data along the time axis
def resample_row(row):
    return signal.resample(row, 200)

data = np.apply_along_axis(resample_row, axis=1, arr=data) # Resample the data to 200 samples
data = np.abs(data)  # Take the absolute value of the data
data = np.log(data + 1)  # Apply logarithmic scaling to the data

# Encode the labels
#label_encoder = LabelEncoder()
#labels = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [49]:
# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=16, validation_data=(x_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Epoch 1/10


ValueError: in user code:

    File "C:\Users\mok93\PycharmProjects\csi_nn\venv\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\mok93\PycharmProjects\csi_nn\venv\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\mok93\PycharmProjects\csi_nn\venv\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\mok93\PycharmProjects\csi_nn\venv\lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\mok93\PycharmProjects\csi_nn\venv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\mok93\PycharmProjects\csi_nn\venv\lib\site-packages\keras\engine

In [53]:
np.shape(X_train)

(5759, 200, 108)

In [13]:
def proccess_label(label):
    data = np.split(read_pcap_from('./train_data/'+ label + '.pcap'), 150)

    for ri in range(np.shape(data)[0]):
        csi = proccess_csi(data[ri])
        path = './train_data/' + label + '/' + label + '_' + str(ri + 1).zfill(3) + '.csv'
        np.savetxt(path, np.asarray(csi), delimiter=",")

In [14]:
proccess_label('empty')
proccess_label('walk')
proccess_label('jump')
proccess_label('wave')